In [ ]:
import * as tslab from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf-8");
tslab.display.html(`<style>${css}</style>`);

# Logic-Solver

# ⚠️ Setup Requirement: Type Definitions

To run this notebook with strict type checking, TypeScript needs type definitions for the `logic-solver` library, which are not included by default.

**Instructions:**
1. Run the code cell below to find the installation path of the `logic-solver` package on your system.
2. Locate the file `logic-solver.d.ts` provided in the current directory.
3. Move or copy this `logic-solver.d.ts` file into the directory path output by the cell below (inside the `logic-solver` folder).
4. Restart the kernel

Once the declaration file is in place, the notebook will compile and run correctly.

In [ ]:
require.resolve('logic-solver');

[Logic-Solver](https://github.com/meteor/logic-solver) is a SAT solver available as an [npm package](https://www.npmjs.com/package/logic-solver). It is a JavaScript wrapper that relies on MiniSat designed for logical constraint satisfaction. We can install it via `npm install logic-solver`.

In [ ]:
import * as Logic from 'logic-solver';

In this Notebook, the [logic-solver](https://github.com/meteor/logic-solver) library is used to efficiently solve propositional logic (SAT) problems.

- **Boolean variables** are implicitly declared by using strings (e.g., `"p"`, `"q"`, `"r"`).
- **Clauses** are constructed using `Logic.or(...)`, with strings serving as literals.
- **Negations** are represented by a preceding minus sign in the string (e.g., `"-q"` for $\neg q$). Alternatively, `Logic.not(...)` can be used, though it often requires the parameter to already be a formula object.
- The problem is built as a set of clauses (Conjunctive Normal Form, CNF) by adding each clause individually using `solver.require(...)`. This corresponds to a large AND over all clauses.
- The solver is initialized with `new Logic.Solver()`.
- The satisfiability check is performed synchronously using `solver.solve()`.
- If the formula is satisfiable, `solve()` returns a solution object, from which the array of true variables can be retrieved using `getTrueVars()`. Otherwise, `null` is returned.

Finally, the formula 
$$ (p \vee \neg q \vee r) \wedge (\neg p \vee q \vee \neg r) \wedge (\neg p \vee \neg q \vee r) \wedge (p \vee q \vee \neg r), $$
which is in conjunctive normal form, is represented by the following code:

In [ ]:
const solver = new Logic.Solver();
solver.require(Logic.or("p", "-q", "r"));
solver.require(Logic.or("-p", "q", "-r"));
solver.require(Logic.or("-p", "-q", "r"));
solver.require(Logic.or("p", "q", "-r"));

In [ ]:
const sol = solver.solve();
if (sol !== null) {
    sol.getMap();
}

This shows that the formula `f` is satisfiable and that the  propositional valuation 
$$ \mathcal{I} = \{ p \mapsto \texttt{False}, q \mapsto \texttt{False}, r \mapsto \texttt{False} \} $$
is a solution for `f`, i.e. we have
$$ \mathcal{I}(\texttt{f}) = \texttt{True}. $$

## Sudoku

In [ ]:
import { RecursiveSet } from 'recursive-set';

type Variable = string;
type NegatedVariable = `-${string}`; 
type Literal = Variable | NegatedVariable;
type Clause = RecursiveSet<Literal>;

The finish mathematician Arto Inkala claims to have created the [hardest sudoku](https://abcnews.go.com/blogs/headlines/2012/06/can-you-solve-the-hardest-ever-sudoku) ever.  It is defined below.

In [ ]:
function createPuzzle(): (number | string)[][] {
  return [
    [ 8 , "*", "*", "*", "*", "*", "*", "*", "*"], 
    ["*", "*",  3 ,  6 , "*", "*", "*", "*", "*"],
    ["*",  7 , "*", "*",  9 , "*",  2 , "*", "*"],
    ["*",  5 , "*", "*", "*",  7 , "*", "*", "*"],
    ["*", "*", "*", "*",  4 ,  5 ,  7 , "*", "*"],
    ["*", "*", "*",  1 , "*", "*", "*",  3 , "*"],
    ["*", "*",  1 , "*", "*", "*", "*",  6 ,  8 ],
    ["*", "*",  8 ,  5 , "*", "*", "*",  1 , "*"],
    ["*",  9 , "*", "*", "*", "*",  4 , "*", "*"]
  ];
}

We use the following variables:
* `Q<r,c,d>` is a Boolean variable stating that the field in row `r` and column `c` holds the digit `d`.
  Here, `r`, `c`, `d` are all elements from the set $\{1,\cdots,9\}$.
    
The function `varName(row, col, digit)` returns a formated string that is interpreted as a variable name.

In [ ]:
function varName(row: number, col: number, digit: number): string {
    return `Q<${row},${col},${digit}>`;
}

In [ ]:
varName(1,2,3);

In [ ]:
function complement(l: Literal): Literal {
    if (l.startsWith('¬')) {
        return l.substring(1);
    } else {
        return '¬' + l;
    }
}

The function `atMostOne(S)` takes a set `S` of propositional variables as its argument.  It returns a set of clauses
expressing the fact that at most one of the variables of `S` is true.

In [ ]:
function atMostOne(S: RecursiveSet<Variable>): RecursiveSet<Clause> {
    const result = new RecursiveSet<Clause>();
    for (const p of S) {
        for (const q of S) {
            if (p < q) {
                const lit1 = complement(p);
                const lit2 = complement(q);
                result.add(new RecursiveSet<Literal>(lit1, lit2));
            }
        }
    }
    return result;
}

The function `atLeastOne(S)` takes a set `S` of propositional variables as its argument.  It returns a set of clauses
expressing the fact that at least one of the variables of `S` is true.

In [ ]:
function atLeastOne(S: RecursiveSet<Variable>): RecursiveSet<Clause> {
    return new RecursiveSet<Clause>(new RecursiveSet<Literal>(...S));
}

The function `exactlyOne(S)` takes a set `S` of propositional variables as its argument.  It returns a set of clauses
expressing the fact that exactly one of the variables of `S` is true.

In [ ]:
function exactlyOne(S: RecursiveSet<Variable>): RecursiveSet<Clause> {
    const atMost = atMostOne(S);
    const atLeast = atLeastOne(S);
    return atMost.union(atLeast);
}

In [ ]:
exactlyOne(new RecursiveSet<Variable>('a', 'b', 'c'));

The function `exactlyOnce` takes an array `L` of pairs of indices as its argument.  The elements of `L` are pairs of the form
`(row, col)`, where both `row` and `col` are elements of the set $\{1, \cdots, 9\}$.
It returns a set of formulas expressing that all Sudoku fields specified by the coordinate pairs in `L` take different digits as values.

In [ ]:
function exactlyOnce(L: Array<[number, number]>): RecursiveSet<Clause> {
    let Clauses = new RecursiveSet<Clause>();
    for (let digit = 1; digit <= 9; digit++) {
        const varNames = L.map(([row, col]) => varName(row, col, digit));
        const vars = new RecursiveSet<Variable>(...varNames);
        const exact = exactlyOne(vars);
        for (const clause of exact) {
            Clauses.add(clause);
        }
    }
    return Clauses;
}

In [ ]:
const result = exactlyOnce(
    Array.from({ length: 9 }, (_, i) => [1, i + 1] as [number, number])
);
for (const clause of result) {
    console.log(`{${[...clause].map(lit => JSON.stringify(lit)).join(', ')}}`);
}

The function `exactlyOneDigit(row, col)` takes integers `row` and `col` as arguments.  These specify the row and column of a field in a Sudoku.  The function returns a set of clauses specifying that exactly one of the variables

* `Q<row,col,1>`, `Q<row,col,2>`, $\cdots$, `Q<row,col,9>`

is `True`.

In [ ]:
function exactlyOneDigit(row: number, col: number): RecursiveSet<Clause> {
    const vars = new RecursiveSet<Variable>();  
    for (let digit = 1; digit <= 9; digit++) {
        vars.add(varName(row, col, digit));
    }
    return exactlyOne(vars);
}

In [ ]:
for (const clause of exactlyOneDigit(1, 1)) {
    console.log(clause.toString()); 
}

The function `constraintsFromPuzzle`  returns a set of clauses stating that the variables corresponding to numbers that are already given in the Sudoku puzzle take the values that are specified.

In [ ]:
function constraintsFromPuzzle(): RecursiveSet<Clause> {
    const Puzzle = createPuzzle();
    const Clauses = new RecursiveSet<Clause>();
    for (let row = 0; row < 9; row++) {
        for (let col = 0; col < 9; col++) {
            const value = Puzzle[row][col];
            if (value !== '*') {
                const v = varName(row + 1, col + 1, value as number);
                const unitClause = new RecursiveSet<Literal>(v);
                Clauses.add(unitClause);
            }
        }
    }
    return Clauses;
}

In [ ]:
for (const clause of constraintsFromPuzzle()) {
    console.log(clause.toString()); 
}

The function `allConstraints` returns a CSP that encodes the given sudoku as a CSP.

In [ ]:
function allConstraints(): RecursiveSet<Clause> {
    const L = [1, 2, 3, 4, 5, 6, 7, 8, 9];
    // 1. Start with constraints from the puzzle
    let Clauses = constraintsFromPuzzle();
    // 2. There is exactly one digit in every field
    for (const row of L) {
        for (const col of L) {
            const digitConstraints = exactlyOneDigit(row, col);
            Clauses = Clauses.union(digitConstraints);
        }
    }
    // 3. All entries in a row are unique
    for (const row of L) {
        const rowCells = L.map(col => [row, col] as [number, number]);
        Clauses = Clauses.union(exactlyOnce(rowCells));
    }
    // 4. All entries in a column are unique
    for (const col of L) {
        const colCells = L.map(row => [row, col] as [number, number]);
        Clauses = Clauses.union(exactlyOnce(colCells));
    }
    // 5. All entries in a 3x3 square are unique
    for (let r = 0; r < 3; r++) {
        for (let c = 0; c < 3; c++) {
            const blockCells: Array<[number, number]> = [];
            for (let row = 1; row <= 3; row++) {
                for (let col = 1; col <= 3; col++) {
                    blockCells.push([r * 3 + row, c * 3 + col]);
                }
            }
            Clauses = Clauses.union(exactlyOnce(blockCells));
        }
    }
    return Clauses;
}

In [ ]:
const clauses = allConstraints();
console.log("--- Clauses with size 1 ---");
for (const clause of clauses) {
    if (clause.size === 1) {
        console.log(clause.toString());
    }
}
console.log("\n--- Clauses with size 9 ---");
for (const clause of clauses) {
    if (clause.size === 9) {
        console.log(clause.toString());
    }
}

In [ ]:
clauses.size

In the code below the clauses are iteratively processed to construct the logical formula for ``logic-solver``. For each clause, the terms are collected, handling negations explicitly by converting negative literals (e.g., `¬Q<1,1,1>`) into `-Q<1,1,1>`.

In [ ]:
const solver = new Logic.Solver();
for (const clause of clauses) {
    const terms: (string | Logic.Formula)[] = [];
    for (const literal of clause) {
        if (literal.startsWith('¬')) {
            const varName = literal.substring(1);
            terms.push("-" + varName);
        } else {
            terms.push(literal);
        }
    }
    solver.require(Logic.or(...terms));
}

In [ ]:
9**3

Even though this Sudoku is modelled using $9^3 = 729$ propositional variables and we have 10551 clauses, logic-solver uses about 15 milliseconds to solve the problem. 

In [ ]:
console.time('Solving');
const sol = solver.solve();
console.timeEnd('Solving');
if (sol === null) {
    throw new Error("No solution found!");
}
const Solution = sol.getTrueVars();

In [ ]:
Solution

## Graphical Representation

In [ ]:
import { display } from 'tslab';
function showSolution(Solution: string[], width: string = '50%'): void {
    const Sudoku = createPuzzle();
    const solutionMap: Record<string, number> = {};
    for (const literal of Solution) {
        const match = literal.match(/^Q<(\d+),(\d+),(\d+)>$/);
        if (match) {
            const row = parseInt(match[1], 10);
            const col = parseInt(match[2], 10);
            const digit = parseInt(match[3], 10);
            solutionMap[`V${row}${col}`] = digit;
        }
    }
    let html = `<table style="width:${width}; border-collapse: collapse; border: 2px solid black; font-family: sans-serif;">`;
    for (let row = 0; row < 9; row++) {
        html += '<tr>';
        for (let col = 0; col < 9; col++) {
            const key = `V${row + 1}${col + 1}`;
            let value = solutionMap[key];
            const original = Sudoku[row][col];
            let cellStyle = "font-weight: normal;";
            if (original !== '*') {
                value = original as number;
                cellStyle = "font-weight: bold;";
            }
            const blockRow = Math.floor(row / 3);
            const blockCol = Math.floor(col / 3);
            const isGray = (blockRow + blockCol) % 2 !== 0;
            const bgColor = isGray ? '#f0f0f0' : '#ffffff';
            let borderStyle = "border: 1px solid #ccc;";
            if ((col + 1) % 3 === 0 && col < 8) borderStyle += "border-right: 2px solid black;";
            if ((row + 1) % 3 === 0 && row < 8) borderStyle += "border-bottom: 2px solid black;";
            html += `<td style="${borderStyle} width:30px; height:30px; text-align:center; font-size:20px; background-color:${bgColor}; ${cellStyle}">${value || ''}</td>`;
        }
        html += '</tr>';
    }
    html += '</table>';
    display.html(html);
}

In [ ]:
showSolution(Solution, "50%");